In [8]:
import cv2
import numpy as np
import time

def filtrar_color_amarillo(imagen):
    # Convertir la imagen a formato HSV (Hue, Saturation, Value)
    hsv = cv2.cvtColor(imagen, cv2.COLOR_BGR2HSV)
    
    # Definir el rango de colores amarillos en HSV
    lower_yellow = np.array([18, 100, 100])
    upper_yellow = np.array([32, 255, 255])
    
    # Aplicar el filtro para detectar solo el color amarillo
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    # Crear una imagen binaria utilizando la máscara
    resultado = cv2.bitwise_and(imagen, imagen, mask=mask)
    
    return mask, resultado


def remove_small_particles(binary_image, min_area):
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a mask to store the filtered particles
    filtered_mask = np.zeros_like(binary_image)
    
    # Iterate through contours
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)
        
        # If the area is larger than the minimum area threshold, keep the contour
        if area >= min_area:
            cv2.drawContours(filtered_mask, [contour], -1, 255, cv2.FILLED)
    
    return filtered_mask

def expand_binary_zone(binary_image, iterations=1):
    # Definir el kernel para la operación de dilatación
    kernel = np.ones((3, 3), np.uint8)
    
    # Aplicar la operación de dilatación
    expanded_image = cv2.dilate(binary_image, kernel, iterations=iterations)
    
    return expanded_image



def shrink_binary_zone(binary_image, iterations=1):
    # Define the kernel for the erosion operation
    kernel = np.ones((3, 3), np.uint8)
    
    # Apply the erosion operation
    shrunk_image = cv2.erode(binary_image, kernel, iterations=iterations)
    
    return shrunk_image

def filtering_1(imagen, mode=0,dif=3):

    mask_amarillo, resultado_amarillo = filtrar_color_amarillo(imagen)
    # Mostrar las imágenes

    # Set the minimum area threshold
    height, width = mask_amarillo.shape
    min_area_threshold = int(height*width*0.00005)

    # Remove small particles from the binary image
    filtered_image = remove_small_particles(mask_amarillo, min_area_threshold)

    if mode!=0:
        cv2.imshow('Original', imagen)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imshow('Máscara Amarilla', mask_amarillo)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    filtered_image=expand_binary_zone(filtered_image,dif)

    if mode!=0:
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    filtered_image=shrink_binary_zone(filtered_image,dif)

    if mode!=0:
        cv2.imshow('Filtered Binary Image', filtered_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    bordes = cv2.Canny(filtered_image, 100, 200)
    if mode!=0:
        cv2.imshow('Bordes', bordes)
        cv2.waitKey(0)
        #cv2.waitKey(0)
        cv2.destroyAllWindows()
    return bordes



In [7]:
import cv2
import numpy as np
import math
def add_border(image, top, bottom, left, right):
    """
    Add borders of specified size to a binary image.

    Parameters:
        image: Input binary image.
        top: Size of the top border.
        bottom: Size of the bottom border.
        left: Size of the left border.
        right: Size of the right border.

    Returns:
        Image with borders added.
    """
    # Define border size for each side
    border_size = (top, bottom, left, right)

    # Create a border filled with zeros
    bordered_image = np.zeros((image.shape[0] + top + bottom, image.shape[1] + left + right), dtype=np.uint8)

    # Copy the original image to the center of the bordered image
    bordered_image[top:top + image.shape[0], left:left + image.shape[1]] = image

    return bordered_image



def linear_transform(image, transformation_matrix, output_size):
    transformed_image = cv2.warpAffine(image, transformation_matrix, output_size, flags=cv2.INTER_NEAREST)
    return transformed_image

def ScalRot_transformation(image,scale_x,scale_y,angle_a_deg):
    angle_a_rad = math.radians(angle_a_deg)

    # Calculate the transformation matrix
    transformation_matrix = np.float32([
        [scale_x * math.cos(angle_a_rad), -scale_y * math.sin(angle_a_rad), 30],
        [scale_x * math.sin(angle_a_rad), scale_y * math.cos(angle_a_rad), 0]
    ])
    # Get the shape of the input image
    height, width = image.shape[:2]
    # Calculate the size of the output image to accommodate all pixels
    corners = np.array([[0, 0], [width, 0], [0, height], [width, height]])
    transformed_corners = cv2.transform(np.array([corners]), transformation_matrix)[0]
    min_x, min_y = np.min(transformed_corners, axis=0)
    max_x, max_y = np.max(transformed_corners, axis=0)
    output_size = (int(max_x - min_x), int(max_y - min_y))
    # Update the transformation matrix to translate the image to keep it within bounds
    transformation_matrix[0, 2] -= min_x
    transformation_matrix[1, 2] -= min_y
    # Apply the affine transformation
    transformed_image = linear_transform(image, transformation_matrix, output_size)

    return transformed_image

In [42]:
def Obj_fun(atributes,BinImage=0,BinKernel=0,mode=0):
    if (type(BinImage)==int or type(BinKernel)==int):
        print('there are no minaes')
        return 0
    [a,px,py,Ax,Ay]=atributes

    if mode!=0:
        cv2.imshow('ker Binary Image', BinKernel)
        cv2.imshow('img Binary Image', BinImage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    NewKer=ScalRot_transformation(BinKernel,Ax,Ay,a)
    height, width = NewKer.shape
    height2=int(height/2)
    width2=int(width/2)

    if mode!=0:
        cv2.imshow('ker Binary Image', NewKer)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


    NewImg=add_border(BinImage,height2,height2,width2,width2)

    if mode!=0:
        cv2.imshow('img Binary Image', NewImg)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    NewImg = NewImg[py:py+height, px:px+width]
    NewImg = cv2.resize(NewImg, (width, height), interpolation=cv2.INTER_NEAREST)

    if mode!=0:
        cv2.imshow('img Binary Image', NewImg)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    diff = cv2.bitwise_and(NewImg, NewKer)
    if mode!=0:
        cv2.imshow('diff Binary Image', diff)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return np.sum(diff)/(255*(Ax+Ay))
    
    

In [46]:
image = cv2.imread('test1.jpg')
kernel = cv2.imread('Rombo.jpg')
a=0
px=1300
py=800
Ax=2
Ay=2.5
atributes=[a,px,py,Ax,Ay]
Obj_fun(atributes,BinImage=filtering_1(image),BinKernel=filtering_1(kernel,dif=1),mode=1)

0.0

In [8]:
class GenAlg:
    def __init__(GA, Npop, Nvar,fun):
        GA.Npop=Npop
        GA.Nvar=Nvar
        GA.population = np.zeros((Npop, Nvar))
        GA.ObjFun=fun
        GA.fitness=np.zeros(Npop)

    def FitEval(GA):
        for i in range(0,GA.Npop):
            GA.fitness[i]=GA.ObjFun(GA.population[i])
            
    

In [12]:
for i in range(1, 7):  # Suponiendo que tienes imágenes de test1.jpg a test10.jpg
    # Formar el nombre de archivo
    filename = f'test{i}.jpg'

    # Cargar la imagen
    image = cv2.imread(filename)

    # Comprobar si la imagen se cargó correctamente
    if image is not None:
        filtering_1(image, mode=1)
    else:
        print(f'No se pudo cargar la imagen {filename}')

In [ ]:
def fun(var):
    return var[0]+var[1]

x=GenAlg(10,2,fun)
print(x.population)
print(x.fitness)
x.FitEval()
print(x.fitness)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [14]:

# Example usage:
# Load your binary image
binary_image = cv2.imread('Rombo.png')
binary_image=filtering_1(binary_image,dif=1)

# Add borders with specified dimensions
bordered_image = add_border(binary_image, top=20, bottom=20, left=30, right=30)

# Display the original and bordered images
cv2.imshow('Original Image', binary_image)
cv2.imshow('Bordered Image', bordered_image)
cv2.waitKey(0)
cv2.destroyAllWindows()